In [1]:
# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# from sklearn.multiclass import OneVsOneClassifier
# Preprocessing and visualization
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Metric functions
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.metrics import accuracy_score

# Models
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors    import KNeighborsClassifier
from sklearn.dummy        import DummyClassifier
from sklearn.tree         import DecisionTreeClassifier

# Model selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# Ignore warnings if they happen, we don't care (that much)
import warnings; warnings.simplefilter('ignore')

# Cross-validation takes a minute, so we will save these models
from sklearn.externals import joblib

In [2]:
df     = pd.read_csv("data/lyrical_genius.csv")

# Remove pop songs, they are all over the place and hurt classification
df = df[(df["Genre"] != "pop")]

# Remove some irrelevant columns
df = df.drop(columns="Unnamed: 0")
df = df.drop(columns="Unnamed: 0.1")

# We go ahead and remove ALL duplicates
df = df.drop_duplicates(subset=["Name","Artist"],keep=False)

# Give each genre a new cool color
genres = df["Genre"].unique()
unique_colors = [
    '#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080'
]
colors = {}
i = 0
for genre in genres:
    colors[genre] = unique_colors[i]
    i+=1
    
# Upsample the amount of occurances of values that don't appear very often
# df = df.append(df[((df["Genre"] != "country") & (df["Genre"] != "edm_dance"))])
extras    = df.copy()
counts    = df["Genre"].value_counts()
max_count = max(df["Genre"].value_counts())
for genre in genres:
    needed = max_count - counts[genre]
    extras = extras.append(df[df["Genre"]==genre].sample(n=needed,replace=True))
df = extras
counts    = df["Genre"].value_counts()
colors_list = [colors[genre] for genre in genres]

df.head()

,Genre,Id,Popularity,Is_Exp,Name,Artist,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time_Signature,Lyrics
0,rnb,3ibKnFDaa3GhpPGlOUj7ff,80,False,Let Me Love You,Mario,0.656,0.578,7,-8.970,0,0.0922,0.2350,0.0,0.1180,0.556,94.514,4,"Mmmm Mmmmm Yeah Mmmmm Yeah, yeah, yeah Mmmm Y..."
1,rnb,2aIdVb8v9KTpEZnftkz2mD,78,False,Buy U a Drank (Shawty Snappin'),T-Pain,0.451,0.550,1,-8.137,1,0.2620,0.0108,0.0,0.0737,0.594,80.001,4,None
2,rnb,7DFnq8FYhHMCylykf6ZCxA,69,False,Yo (Excuse Me Miss),Chris Brown,0.536,0.612,4,-5.847,1,0.2720,0.1190,0.0,0.2090,0.570,86.768,4,Let’s get ONE. THING. STRAIGHT! Certain shit ...
3,rnb,39YovPslPCXbFYhlYjsZ2Y,67,False,Don't Mess With My Man,Nivea,0.879,0.730,11,-4.369,0,0.1640,0.1140,0.0,0.2410,0.885,99.925,4,"] Uh, I like it baby, uh Uh, one time for the..."
4,rnb,3LmvfNUQtglbTrydsdIqFU,72,False,We Belong Together,Mariah Carey,0.838,0.469,0,-7.992,1,0.0835,0.0358,0.0,0.0928,0.778,139.975,4,"Ooh, oh oh Sweet love, yeah I didn't mean i..."


In [3]:
# Split data into data frames of the right type
x_cols    = ["Is_Exp","Danceability","Energy","Key","Loudness","Mode","Speechiness","Acousticness","Instrumentalness","Liveness","Valence","Tempo","Time_Signature"]
y_cols    = ["Genre"]
meta_cols = ["Id","Popularity","Name","Artist"]

X,y,meta = df[x_cols],df[y_cols].iloc[:,0],df[meta_cols]

In [4]:
# Scale the data and fit run 2D PCA on it
scaler   = StandardScaler()
scaled_X = scaler.fit_transform(X)
pca = PCA(n_components=2)
prin_comp = pca.fit_transform(scaled_X)
prin_df   = pd.DataFrame(data=prin_comp, columns=["PC1","PC2"])

In [ ]:
# Split 
X_train, X_test, y_train, y_test = train_test_split(scaled_X,y, test_size=.2, random_state=1234, stratify=y)

In [ ]:
clf = LogisticRegressionCV(cv=5, random_state=1234, multi_class="multinomial")
clf.fit(X_train,y_train)

# joblib.dump(clf, "saved_logistic.pkl")
y_pred = clf.predict(X_test)
training = clf.score(X_train, y_train)
testing  = clf.score(X_test,  y_test)
print("Training Accuracy: {}".format(training))
print("Testing  Accuracy: {}".format(testing))
print(genres)
print(confusion_matrix(y_test,y_pred,labels=genres))
print(classification_report(y_test, y_pred)) 

Training Accuracy: 0.7094246698007611
Testing  Accuracy: 0.7153088630259624
['rnb' 'hiphop' 'country' 'classical' 'edm_dance' 'rock']
[[279 111  85   0  55  29]
 [ 78 455   9   0  13   4]
 [ 41   5 385   6  22  99]
 [  2   1  17 529   7   3]
 [ 38  21  33   6 390  70]
 [ 25   8  92   6  68 359]]
              precision    recall  f1-score   support

   classical       0.97      0.95      0.96       559
     country       0.62      0.69      0.65       558
   edm_dance       0.70      0.70      0.70       558
      hiphop       0.76      0.81      0.78       559
         rnb       0.60      0.50      0.55       559
        rock       0.64      0.64      0.64       558

   micro avg       0.72      0.72      0.72      3351
   macro avg       0.71      0.72      0.71      3351
weighted avg       0.71      0.72      0.71      3351



In [ ]:
for i in range(len(clf.classes_)):
    print(clf.classes_[i])
    todos      = [(clf.coef_[i][j],list(X)[j]) for j in range(len(list(X)))]
    todos.sort(key=lambda x: abs(x[0]))
    print(todos[0])
    print(todos[1])
    
    print(todos[-2])
    print(todos[-1])
    

classical
(-0.06138596065807699, 'Liveness')
(-0.06612078048195241, 'Tempo')
(-1.7133699290422393, 'Danceability')
(-2.480090556672996, 'Loudness')
country
(0.03694284851893798, 'Liveness')
(0.05035586531135824, 'Tempo')
(-0.950302992058766, 'Speechiness')
(-1.215232711461041, 'Is_Exp')
edm_dance
(0.0049253215810497585, 'Acousticness')
(-0.03695437589060417, 'Key')
(-1.1486309309998721, 'Valence')
(1.188274447554809, 'Instrumentalness')
hiphop
(0.04531681419457953, 'Key')
(-0.09608239654457466, 'Liveness')
(-1.354119999534468, 'Instrumentalness')
(2.10360817661885, 'Danceability')
rnb
(-0.010929338150370088, 'Key')
(0.04289658365926415, 'Valence')
(0.7804472430964925, 'Danceability')
(-1.2258601776303388, 'Energy')
rock
(-0.00044547980717340687, 'Key')
(-0.01201091015555885, 'Time_Signature')
(-0.8606024823072914, 'Speechiness')
(-1.5398786534065008, 'Danceability')


In [ ]:
def best_classifier(X, y, t_clf, params) :
    """
    Sweeps different settings for the hyperparameters of a Decision Tree classifier,
    calculating the k-fold CV performance for each setting and metric,
    then selects the hyperparameters that maximize the average performance for each metric.
    """

    
    
    clf  = GridSearchCV(t_clf, params, cv=5,scoring= "accuracy")
    
    clf.fit(X,y)
    
    return clf.best_estimator_

In [ ]:
weights = ["uniform","distance"]
pvals   = [1,2]
n_neigh = range(1,40,2)
params  = {
    "n_neighbors": range(1,40,2),
    "weights": weights,
    "p": pvals,

}
knn = KNeighborsClassifier()
clf = best_classifier(X_train,y_train,knn,params)

joblib.dump(clf, "saved_knn.pkl")

y_pred = clf.predict(X_test)
training = clf.score(X_train, y_train)
testing  = clf.score(X_test,  y_test)
print("Training Accuracy: {}".format(training))
print("Testing  Accuracy: {}".format(testing))
print(confusion_matrix(y_test,y_pred,labels=genres))
print(classification_report(y_test, y_pred)) 

In [ ]:
params = {
    "max_depth": range(5,21),
    "min_samples_leaf": range(1,15),
    "criterion": ["gini","entropy"]
}
t = DecisionTreeClassifier()
print("traing dtre...")
DTree = best_classifier(X_train, y_train, t, params)

print("saving dtree...")
joblib.dump(DTree, "saved_dtree.pkl")
print("done.")
# predict genres of test data
accuracy = DTree.score(X_test,y_test)
y_pred = clf.predict(X_test)
training = DTree.score(X_train, y_train)
testing  = DTree.score(X_test,  y_test)
print("Training Accuracy: {}".format(training))
print("Testing  Accuracy: {}".format(testing))
print(confusion_matrix(y_test,y_pred,labels=genres))
print(classification_report(y_test, y_pred))

In [ ]:
importance = DTree.feature_importances_
d_feats      = list(X)
todos      = [(importance[i],d_feats[i]) for i in range(len(d_feats))]
todos.sort(key=lambda x: x[0],reverse=True)

In [ ]:
todos

In [ ]:
# compare to stratified dummy classifier
dummy = DummyClassifier(strategy='stratified')

joblib.dump(dummy, "saved_dummy.pkl")

dummy.fit(X_train,y_train)
dummy_accuracy = dummy.score(X_test,y_test)
print( "Dummy classifier accuracy is" )
print(dummy_accuracy)